In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import time  # Import the time module to measure elapsed time

# Define the list of crypto symbols
symbols = [
    'ADIN', 'ALK', 'ALKB', 'AMEH', 'APTK', 'ATPP', 'AUMK', 'BANA', 'BGOR', 'BIKF', 'BIM', 'BLTU',
    'CBNG', 'CDHV', 'CEVI', 'CKB', 'CKBKO', 'DEBA', 'DIMI', 'EDST', 'ELMA', 'ELNC', 'ENER', 'ENSA', 
    'EUHA', 'EUMK', 'EVRO', 'FAKM', 'FERS', 'FKTL', 'FROT', 'FUBT', 'GALE', 'GDKM', 'GECK', 'GECT', 
    'GIMS', 'GRDN', 'GRNT', 'GRSN', 'GRZD', 'GTC', 'GTRG', 'IJUG', 'INB', 'INHO', 'INOV', 'INPR', 
    'INTP', 'JAKO', 'JUSK', 'KARO', 'KDFO', 'KJUBI', 'KKST', 'KLST', 'KMB', 'KMPR', 'KOMU', 'KONF', 
    'KONZ', 'KORZ', 'KPSS', 'KULT', 'KVAS', 'LAJO', 'LHND', 'LOTO', 'LOZP', 'MAGP', 'MAKP', 'MAKS', 
    'MB', 'MERM', 'MKSD', 'MLKR', 'MODA', 'MPOL', 'MPT', 'MPTE', 'MTUR', 'MZHE', 'MZPU', 'NEME', 
    'NOSK', 'OBPP', 'OILK', 'OKTA', 'OMOS', 'OPFO', 'OPTK', 'ORAN', 'OSPO', 'OTEK', 'PELK', 'PGGV', 
    'PKB', 'POPK', 'PPIV', 'PROD', 'PROT', 'PTRS', 'RADE', 'REPL', 'RIMI', 'RINS', 'RZEK', 'RZIT', 
    'RZIZ', 'RZLE', 'RZLV', 'RZTK', 'RZUG', 'RZUS', 'SBT', 'SDOM', 'SIL', 'SKON', 'SKP', 'SLAV', 
    'SNBT', 'SNBTO', 'SOLN', 'SPAZ', 'SPAZP', 'SPOL', 'SSPR', 'STB', 'STBP', 'STIL', 'STOK', 'TAJM', 
    'TBKO', 'TEAL', 'TEHN', 'TEL', 'TETE', 'TIKV', 'TKPR', 'TKVS', 'TNB', 'TRDB', 'TRPS', 'TRUB', 
    'TSMP', 'TSZS', 'TTK', 'TTKO', 'UNI', 'USJE', 'VARG', 'VFPM', 'VITA', 'VROS', 'VSC', 'VTKS', 
    'ZAS', 'ZILU', 'ZILUP', 'ZIMS', 'ZKAR', 'ZPKO', 'ZPOG', 'ZUAS'
]

BASE_URL = "https://www.mse.mk/mk/stats/symbolhistory/{symbol}"
HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Content-Type": "application/x-www-form-urlencoded",
}

def fetch_data(symbol, start_date, end_date):
    # Make a request to fetch data for a given symbol and date range
    payload = {
        "FromDate": start_date,
        "ToDate": end_date,
        "Code": symbol,
    }
    url = BASE_URL.format(symbol=symbol)
    response = requests.post(url, data=payload, headers=HEADERS)
    
    if response.status_code != 200:
        print(f"Failed to fetch data for {symbol}")
        return None
    return response.text

def parse_table(html_content):
    # Parse the HTML content and extract table data into a DataFrame
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find("table", {"id": "resultsTable"})
    
    if not table:
        print("No data table found.")
        return None

    # Extract headers
    headers = [header.text.strip() for header in table.find_all("th")]

    # Extract rows
    rows = []
    for row in table.find_all("tr")[1:]:  # Skip header row
        columns = row.find_all("td")
        row_data = [col.text.strip() for col in columns]
        rows.append(row_data)

    # Create DataFrame
    df = pd.DataFrame(rows, columns=headers)
    return df

def save_to_csv(df, symbol, year):
    # Save data to a CSV file named by symbol and year
    filename = f"{symbol}_{year}.csv"
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

def scrape_data_for_all_symbols():
    # Start the timer
    start_time = time.time()

    # Loop through each symbol and year range
    start_year = 2014
    end_year = datetime.now().year
    
    for symbol in symbols:
        for year in range(start_year, end_year + 1):
            print(f"Fetching data for {symbol} for the year {year}...")
            start_date = f"01/01/{year}"
            end_date = f"31/12/{year}"
            html_content = fetch_data(symbol, start_date, end_date)
            
            if html_content:
                df = parse_table(html_content)
                
                if df is not None:
                    save_to_csv(df, symbol, year)

    # End the timer and calculate elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Scraping completed in {elapsed_time:.2f} seconds.")

if __name__ == "__main__":
    scrape_data_for_all_symbols()
